[[source]](../api/alibi.explainers.html#alibi.explainers.PartialDependence)

# Partial Dependence

## Overview

The partial dependence (PD) plot proposed by [J.H. Friedman (2001)](https://projecteuclid.org/journals/annals-of-statistics/volume-29/issue-5/Greedy-function-approximation-A-gradient-boostingmachine/10.1214/aos/1013203451.full) , is a method of visualizing the marginal effect that one or two features have on the predicted outcome of a machine learning model. By inspecting the PD plots, one can understand whether the relation between a feature/pair of features is, for example, a simple linear or quadratic relation, whether it presents a monotonically increasing or decreasing trend, or reveal a more complex response.

Before diving into the mathematical formulation of the PD, we first introduce some notation. Consider $\mathcal{F}$ to be the set of all features, $S$ be a set of features of interest (i.e. $S \subseteq \mathcal{F}$) that we want to compute the marginal effect for, and $C$ be their complement (i.e. $C = \mathcal{F} \setminus S$). It is important to note that the subset $S$ is not only restricted to one or two features as mentioned in the previous paragraph, but can be any subset of the set $\mathcal{F}$. In practice, though, due to visualization reasons, we will not analyze more than two features at the time.

Given a black-box model, $f$, we are ready to define the partial dependence for a set of features $S$ as:

$$
    f_{S}(x_S) = \mathbb{E}_{X_C} [f(x_S, X_C)] = \int f(x_S, X_C) d\mathbb{P}(X_C),
$$

where we denoted random variables by capital letters (e.g., $X_C$), realizations by lowercase letters (e.g. $x_S$), and $\mathbb{P}(X_C)$ the probability distribution/measure over the features set C. 

In practice, to approximate the integral above, we rely on Monte Carol method, computing the average over a reference dataset. Formally, let us consider a reference dataset $\mathcal{X} = \{x^{(1)}, x^{(2)}, ..., x^{(n)}\}$. The PD for the set $S$ can be approximated as:

$$
f_{S}(x_S) = \frac{1}{n} \sum_{i=1}^{n}f(x_S, x_{C}^{(i)}).
$$

In simple words, to compute the marginal effect of the feature values $x_S$, we query the model on synthetic instance created from the concatenation of $x_S$ with the feature values $x_C^{(i)}$ from the reference dataset, and averaging the model responses. Already that from the computation we can identify a major limitation of the method. The PD computation assumes feature independence (i.e. feature not correlated) which is a strong and quite restrictive assumption and usually does not hold in practice. A classic example of positively correlated features is *height* and *weight*. If $S = \{\text{height}\}$ and $C=\{\text{weight}\}$, the independence assumption will create unrealistic synthetic instance, by combining values of *height* and *weight* that do not correlated (e.g. $\text{height} = 1.85\text{m}$ - hight of an adult - and $\text{weight} = 30 \text{kg}$ - weight of child). Such limitation can be addressed by ALE.

Although the ALE can handle correlated features, the PD still have some advantages beyond their simple and intuitive definition. The PD can directly be extended to categorical features. For each category of a feature, one can compute the PD by setting all data instance to have the same category and following the same averaging strategy over the reference dataset (i.e. replace features $C$ with feature values from the reference, compute the response, and average all the responses). Note that ALE requires by definition the feature values to be ordinal, which might not be the case for all categorical features. Depending on the context, there exists some methods that allow the ALE to be extended to categorical feature, for which we recommend the [ALE chapter](https://christophm.github.io/interpretable-ml-book/ale.html) from [Interpretable machine learning](https://christophm.github.io/interpretable-ml-book/) book, C. Molnar(2020) as further reading.

We will continue to discuss the simple case of linear regression. Note that for linear regression, the PD will always show a linear relationship between the features and the response. To formally prove the linear relationship, consider the following linear regression model:

$$
f(x) = \beta_0 + \beta_1 x_1 + ... \beta_{|\mathcal{F}|} x_{|\mathcal{F}|}.
$$

Without loss of generality, we can assume the the features of interest come first in the equation above and the rest of the features at the end. We can rewrite the above equation as follows:

$$
f(x) = \beta_0 + \beta_1 x_{S_1} + ... + \beta_{|S|} x_{S_{|S|}} + \beta_{|S| + 1} x_{C_{1}} + ... + \beta_{|S| + |C|} x_{C_{|C|}},
$$

where $S_{i}$ and $C_{i}$ represent features in $S$ and $C$, respectively.

Following the definition of PD, we obtain:

\begin{equation}
\begin{aligned}
    & f_S(x_{S}) = \mathbb{E}_{X_C}[f(x_S, X_C)] \\
    &  =  \mathbb{E}_{X_C}[\beta_0 + \beta_1 x_{S_1} + ... + \beta_{|S|} x_{S_{|S|}} + \beta_{|S| + 1} x_{C_{1}} + ... + \beta_{|S| + |C|} x_{C_{|C|}}] \\
    & = \beta_0 + \beta_1 x_{S_1} + ... + \beta_{|S|} x_{S_{|S|}} + \mathbb{E}_{X_C}[x_{C_{1}} + ... + \beta_{|S| + |C|} x_{C_{|C|}}] \\
    & = \beta_0 + \beta_1 x_{S_1} + ... + \beta_{|S|} x_{S_{|S|}} + K_C \\
    & = (\beta_0 + K_C) + \beta_1 x_{S_1} + ... + \beta_{|S|} x_{S_{|S|}},
\end{aligned}
\end{equation}

where $K_C = \mathbb{E}_{X_C}[x_{C_{1}} + ... + \beta_{|S| + |C|} x_{C_{|C|}}]$.
Thus we can see that the PD $f_S(X_S)$ is linear in the features in $S$.

For further readings, we strongly recommend the [PDP chapter](https://christophm.github.io/interpretable-ml-book/pdp.html) from the [Interpretable machine learning](https://christophm.github.io/interpretable-ml-book/) book, C. Molnar (2020).

The PD plot shows the marginal effect of a features and thus is a global method (i.e., it does not focus on a particular instance, but average the response over multiple synthetic data instances). The global nature of the PD hides inevitably some heterogeneous effect by feature interactions. To reveal the heterogeneous relationship, one can plot the individual conditional expectation (ICE). Following the definitions from [ICE chapter], C. Molnar (2020), an ICE plot displays the dependence of the prediction on a feature (i.e. due to visualization constraints it is restricted to a single feature) for each instance separately which results in a plot per data instance. Formally, given the reference dataset $\mathcal{X}=\{(x_{S}^{(i)}, x_{C}^{(i)}\}_{i=1}^{N}$ and a black box-model $f$, the ICE plot will display a curve $f^{(i)}$ plotted against $\{x_{S}^{(j)}\}_{j=1}^{N}$ while keeping $x_{C}^{(i)}$ fixed.

For further readings, we strongly recommend the [ICE chapter](https://christophm.github.io/interpretable-ml-book/ice.html) from the [Interpretable machine learning](https://christophm.github.io/interpretable-ml-book/) book, C. Molnar (2020).

## Usage

To initialize the explainer with a `sklearn` model one can directly pass the estimator and optionally a list of feature names, target names and categorical names for interpretation and specification of the categorical features

```python
from alibi.explainers import PartialDependence
pd = PartialDependence(predictor=predictor,
                       feature_names=feature_names,
                       categorical_names=categorical_names,
                       target_names=target_names)
```

Note that the direct support of the `sklearn` is a consequence of reusing some functionalities provided by `sklearn` when building the `alibi` PD explainer.

In addition, the `alibi` PD explainer supports any black-box model by simply providing the prediction function of the model. To use a prediction function, additional meta data regarding the prediction function should be provided through the `predictor_kw` argument.

For a classifier predicting with 3 classes, the initialization is:

```python
pd = PartialDependence(predictor=predictor_fn, 
                       feature_names=feature_names,
                       categorical_names=categorical_names,
                       target_names=target_names,
                       predictor_kw={
                           'predictor_type': 'classifier', 
                           'prediction_fn': 'predict_proba',   # can be 'decision_function'
                           'num_classes': 3
                       })
```

For a regressor, the initialization is:

```python
pd = PartialDependence(predictor=predictor_fn, 
                       feature_names=feature_names,
                       categorical_names=categorical_names,
                       target_names=target_names,
                       predictor_kw={
                           'predictor_type': 'regressor', 
                           'prediction_fn': 'predict',   # can be 'decision_function'
                       })
```

Following the initialization, we can produce an explanation given a reference dataset $X$:

```python
exp = explainer.explain(X=X,
                        features_list=feature_list,
                        kind='average',
                        method='brute')
```

Multiple arguments can be provided to the explain method:

- `X` - An `N x F` reference tabular dataset used to calculate partial dependence curves. This is typically the training dataset or a representative sample.

- `features_list` - An optional list of features or pairs of features for which to calculate the partial dependence for.If not provided, the partial dependence will be computed for every single features in the dataset.

- `response_method` - Specifies the prediction function to be used. For a classifier it specifies whether to use the `predict_proba` or the `decision_function`. For a regressor, the parameter is ignored. If set to `auto`, the `predict_proba` is tried first, and if not supported then it reverts to `decision_function`. Note that if `method='recursion'`, the prediction function always uses `decision_function`.

- `method` - The method used to calculate the average predictions

    * ``'recursion'`` - a faster alternative only supported by some tree-based model. For a classifier, the 
    target response is always the decision function and NOT the predicted probabilities. Furthermore, since
    the ``'recursion'`` method computes implicitly the average of the individual conditional expectation
    (ICE) by design, it is incompatible with ICE and the `kind` parameter must be set to ``'average'``.
    Check the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.partial_dependence.html#sklearn.inspection.partial_dependence) for a list of supported tree-based classifiers.

    * ``'brute'`` - supported for any black-box prediction model, but is more computationally intensive.
    
    * ``'auto'`` - uses ``'recursion'`` if the `predictor` supports it. Otherwise, uses the ``'brute'`` method.

- `kind` - If set to ``'average'``, then only the partial dependence (PD) averaged across all samples from the dataset is returned. If set to ``individual``, then only the individual conditional expectation (ICE) is returned for each individual from the dataset. Otherwise, if set to ``'both'``, then both the PD and the ICE are returned. Note that for the faster ``method='recursion'`` option the only compatible parameter value is ``kind='average'``. To plot the ICE, consider using the more computation intensive ``method='brute'``.

- `percentiles` - Lower and upper percentiles used to create extreme values which can potential remove outliers in low density regions. The values must be in [0, 1].

- `grid_resolution` - Number of equidistant points to split the range of each target feature. Only applies if the number of unique values of a target feature in the reference dataset `X` is less than the `grid_resolution` value.

- `grid_points` - Custom grid points. Must be a `dict` where the keys are the target features indices and the values are monotonically increasing `numpy` arrays defining the grid points for numerical feature, and a subset of categorical feature values for a categorical feature. If the `grid_points` are not specified, then the grid will be constructed based on the unique target feature values available in the reference dataset `X`, or based on the `grid_resolution` and `percentiles` (check `grid_resolution` to see when it applies). For categorical features, the corresponding value in the `grid_points` can be specified either as `numpy` array of strings or `numpy` array of integers corresponding the label encodings. Note that the label encoding must match the ordering of the values provided in the `categorical_names`.

The result exp in `Explanation` object which contains the following data-related attributes:

- `feature_values` - A list of arrays or list of arrays containing the evaluation points for each explained feature passed in the `feature_list` argument (see `explain` method).
 
- `feature_names` - A list of strings or tuples of string containing the names associated with the explained features elements from `feature_values`.

- `feature_deciles` - a list of arrays (one for each numerical features) of the explained feature deciles.

- `pd_values` - a list of arrays of PD values (one for each feature/pair of features). Each array has a shape of `T x (V1 x V2 x ...)`, where `T` is the number of target outputs, and `Vi` is the number of evaluation points for the corresponding feature `fi`.

- `ice_values` - a list of arrays of ICE values (one for each feature/pair of feature). Each array has a shape of `T x N x (V1 x V2 x ...)`, where `T` is the number of target outputs, `N` is the number of instances in the reference dataset, and `Vi` is the number of evaluation points for the corresponding feature `fi`.

- `meta` - Dictionary containing the following metadata:

    - `feature_names`, `categorical_names`, `target_names` - See `__init__` method.

    - `response_method`, `method` - See `explain` method. The value might have changed based on some internal logic.

    - `kind` - See `explain` method.

Plotting the `pd_values` and `ice_values` against `exp_feature_values` recovers the PD and the ICE plots, respectively. For convenience we included a plotting function `plot_pd` which automatically produces PD and ICE plots using `matplotlib` and `seaborn`.

```python
from alibi.explainers import plot_pd
plot_pd(exp)
```

The following are the one way PD plots for a random forest regression trained on the [Bike rental](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) dataset (see worked [example](../examples/pdp_regression_bike.ipynb)).

<img src="pdp_one_bike.png" alt="PD plots, one feature, Bike rental dataset." width="1000"/>

The following are the ICE plots for a random forest regression trained on the [Bike rental](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) dataset (see worked [example](../examples/pdp_regression_bike.ipynb)).

<img src="ice_bike.png" alt="ICE plots, one feature, Bike rental dataset." width="1000"/>


The following are the two way PD plots for a random forest regression trained on the [Bike rental](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset) dataset (see worked [example](../examples/pdp_regression_bike.ipynb)).

<img src="pdp_two_bike.png" alt="PD plots, two features, Bike rental dataset." width="1000"/>



## Examples

[PD, ICE regression example (Bike rental)](../examples/pdp_regression_bike.ipynb)